In [1]:
import os

import numpy as np
import pandas as pd

import torch
import torch.nn.functional as F
import torch.optim as optim


from datetime import datetime
from torch import nn
from torch_geometric.data import Data, DataLoader
from torch_geometric.nn import MessagePassing
from torch_scatter import scatter
from torch.utils.tensorboard import SummaryWriter

from DataClasses import lmdb_dataset, Dataset
from ModelFunctions import train, evaluate, inference

In [2]:
#вызывается каждый раз, когда датасет отдаёт элемент (систему)
#делаем из данных матрицу векторов-атомов, список рёбер (edge_index) и матрицу векторов-рёбер; надо писать свою функцию для каждой сети
def preprocessing(system):
    
    atom_embeds = torch.cat(atom_features, 1)
    
    edge_index = system['edge_index_new'].long()

    edges_embeds = torch.cat(edge_features, 1)
    
    return Data(x=atom_embeds.to(device), edge_index=edge_index.to(device), edge_attr=edges_embeds.to(device))

$$
\mathbf{x}_i^{(k)} = \gamma^{(k)} \left( \mathbf{x}_i^{(k-1)}, \square_{j \in \mathcal{N}(i)} \, \phi^{(k)}\left(\mathbf{x}_i^{(k-1)}, \mathbf{x}_j^{(k-1)},\mathbf{e}_{j,i}\right) \right)
$$

Гамма лежит в апдейт, квадратик в aggr, а фи в месседж; в этом примере квадратик -- суммирование

In [ ]:
class EmbeddingBlock(torch.nn.Module):
    def __init__(self, embedding_dim, embedded_hidden, element_number, fc1_input,  fc1_to_fc2_hidden, fc2_embedded_hidden, fc3_out, activation):
        super().__init__()
        self.max_number_of_elements = element_number
        
        self.embeding = nn.Embedding(num_embeddings=self.max_number_of_elements, embedding_dim=embedding_dim)

        nn.init.uniform_(self.source_embed.weight.data, -0.0001, 0.0001)
        nn.init.uniform_(self.target_embed.weight.data, -0.0001, 0.0001)

        self.embedding_dim = embedding_dim
        self.fc_embedding = nn.Linear(2 * self.embedding_dim, embedded_hidden)
        
        self.fc1 = nn.Linear(fc1_input, fc1_to_fc2_hidden)
        self.fc2 = nn.Linear(fc1_to_fc2_hidden, fc2_embedded_hidden*self.embedding_dim)

        self.fc3 = nn.Linear(fc2_embedded_hidden, fc3_out)

        self.softmax = nn.Softmax(dim=1)

        self.activation = activation

    # def buildEmbed(x, connectivity):
    #     for i in 

    def forward(self, batch):
        
        x = torch.tensor(batch['x'], dtype=torch.long)
        edge_index = torch.tensor(batch['edge_index'], dtype=torch.long)
        edge_attr = torch.tensor(batch['edge_attr'], dtype=torch.long)

        source_embedding = self.embeding(x[edge_index[0]])
        target_embedding = self.embeding(x[edge_index[1]])

        x = self.fc1(x)
        x = self.activation(x)
        x = self.fc2(x)
        x = self.activation(x)
        x = (x.view(-1, self.num_embedding_basis, self.mid_hidden_channels)
        ) * (embedding.view(-1, self.num_embedding_basis, 1))
        x = torch.sum(x, dim=1)
        x = self.fc3(x)

In [ ]:

#model
model = ConvNN(dim_atom=training_set[0][0]['x'].shape[1], dim_edge=training_set[0][0]['edge_attr'].shape[1])

#optimizer and loss
optimizer = optim.AdamW(model.parameters(), lr=lr)
criterion = nn.L1Loss()

#переносим на куду если она есть
model = model.to('cpu')
criterion = criterion.to('cpu')

logfile_str = {
    "lr": "0.001",
    "smearing" : "smearing"
}

#граф модели
trace_system = dict(list(next(iter(training_generator))[0]))
writer.add_graph(model, trace_system)
writer.add_text(timestamp, str(logfile_str))

In [3]:
class GConv(MessagePassing):
    def __init__(self):
        super(GConv, self).__init__(aggr='add')  # "Add" aggregation

    def forward(self, batch):
        x = batch['x']
        edge_index = batch['edge_index']
        edge_attr = batch['edge_attr']
        
        # x has shape [N -- количество атомов в системе(батче), in_channels -- размерность вектора-атома]
        # edge_index has shape [2, E] -- каждое ребро задаётся парой вершин

        # Start propagating messages. 
    
        return self.propagate(edge_index, x=x, edge_attr=edge_attr, size=None)

    def message(self, x, x_i, x_j, edge_attr):
        # your function
        pass
        
    def update(self, aggr_out, x, edge_attr, edge_index):        
        #your function
        pass

In [4]:
class ConvNN(nn.Module):
    
    def __init__(self):
        
        super().__init__()          
        self.conv = GConv()
        
    def forward(self, batch):
        #your function
        pass

In [5]:
#config
batch_size = 50
num_workers = 0

features_cols = ['atomic_numbers', 'edge_index_new', 'distances_new', 
                 'contact_solid_angles', 'tags', 'voronoi_volumes', 'spherical_domain_radii']

target_col = 'y_relaxed'
lr = 0.001
epochs = 20

In [6]:
#чтобы тензор по умолчанию заводился на куде
if torch.cuda.is_available():
    torch.set_default_tensor_type('torch.cuda.FloatTensor')
    print('cuda')

In [7]:
#set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')  
print(device)

cpu


In [8]:
#инициализируем тренировочный датасети и тренировочный итератор
train_dataset_file_path= os.path.expanduser("../../ocp_datasets/data/is2re/10k/train/data_mod2.lmdb")

training_set = Dataset(train_dataset_file_path, features_cols, target_col, preprocessing=preprocessing)
training_generator = DataLoader(training_set, batch_size=batch_size, num_workers=num_workers)

In [9]:
#инициализируем валидационный датасет и валидационный итератор
val_dataset_file_path = os.path.expanduser("../../ocp_datasets/data/is2re/all/val_ood_both/data_mod2.lmdb")

valid_set = Dataset(val_dataset_file_path, features_cols, target_col, preprocessing=preprocessing)
valid_generator = DataLoader(valid_set, batch_size=batch_size, num_workers=num_workers)

In [10]:
try:
    lmdb_dataset(train_dataset_file_path).describe()
except:
    pass

item: 0
edge_index:............... [2, 2964]
pos:......................   [86, 3]
cell:..................... [1, 3, 3]
atomic_numbers:...........      [86]
natoms:...................        86
cell_offsets:............. [2964, 3]
force:....................   [86, 3]
distances:................    [2964]
fixed:....................      [86]
sid:......................   2472718
tags:.....................      [86]
y_init:...................    6.2825
y_relaxed:................   -0.0256
pos_relaxed:..............   [86, 3]
voronoi_volumes:..........      [86]
voronoi_surface_areas:....      [86]
spherical_domain_radii:...      [86]
cell_offsets_new:......... [1214, 3]
distances_new:............    [1214]
contact_solid_angles:.....    [1214]
direct_neighbor:..........    [1214]
edge_index_new:........... [2, 1214]


In [12]:
#model
model = ConvNN(dim_atom=training_set[0][0]['x'].shape[1], dim_edge=training_set[0][0]['edge_attr'].shape[1])

#optimizer and loss
optimizer = optim.AdamW(model.parameters(), lr=lr)
criterion = nn.L1Loss()

#переносим на куду если она есть
model = model.to(device)
criterion = criterion.to(device)

ValueError: optimizer got an empty parameter list

In [13]:
timestamp = str(datetime.now().strftime("%Y-%m-%d-%H-%M-%S"))

print(timestamp)

2021-09-14-12-17-59


In [ ]:
#tensorboard writer, при первом запуске надо руками сделать папку для логов

# server
#log_folder_path = "../../ocp_results/logs/tensorboard/out_base_model"

# colab
# log_folder_path = "/content/drive/MyDrive/ocp_results/logs/tensorboard/out_base_model"

# user_specific 
log_file_path = "../logs/tensorboard_airi"

writer = SummaryWriter(log_file_path + '/' + timestamp)

In [ ]:
%%time
logfile_str = {
    "train_dataset_file_path": train_dataset_file_path,
    "val_dataset_file_path": val_dataset_file_path,
    "features_cols": features_cols,
    "target_col": target_col,
    "batch_size": batch_size,
    "num_workers": num_workers,
    "epochs": epochs,
    "lr": lr
}

#граф модели
trace_system = dict(list(next(iter(training_generator))[0]))
writer.add_graph(model, trace_system)
writer.add_text(timestamp, str(logfile_str))

## Training

In [ ]:
%%time
loss = []
loss_eval = []

print(timestamp)
print(f'Start training model {str(model)}')
for i in range(epochs):
    loss.append(train(model, training_generator, optimizer, criterion, epoch=i, writer=writer, device=device))
    loss_eval.append(evaluate(model, valid_generator, criterion, epoch=i, writer=writer, device=device))